# 🎭 다중 모델 앙상블 - 5개 모델 통합
> PRD 계획에 따른 5개 모델 앙상블 + TTA 전략

**목표 성능**: ROUGE-F1 75-80

In [1]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent.parent  # 3번만 parent 사용!

# 다른 프로젝트 경로 제거하고 현재 프로젝트 경로만 추가
sys.path = [p for p in sys.path if 'computer-vision-competition' not in p]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project Root: {project_root}")
print(f"Current Dir: {notebook_dir}")

# 필요한 라이브러리 임포트
import yaml
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from typing import List, Dict
import matplotlib.pyplot as plt
import wandb

# 커스텀 모듈 임포트
from src.logging.notebook_logger import NotebookLogger
from src.utils.gpu_optimization.team_gpu_check import check_gpu_tier
from src.utils.visualizations.training_viz import TrainingVisualizer

print("Libraries imported successfully!")

Project Root: /home/ieyeppo/AI_Lab/natural-language-processing-competition
Current Dir: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH
✅ 나눔고딕 폰트 로드 성공
Libraries imported successfully!


In [2]:
# 설정 파일 로드
config_path = notebook_dir / 'configs' / 'config_multi_model.yaml'

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# 활성화된 모델 확인
enabled_models = [name for name, cfg in config['ensemble_models'].items() if cfg['enabled']]
print(f"Enabled Models ({len(enabled_models)}):")
for model_name in enabled_models:
    weight = config['ensemble_models'][model_name]['weight']
    print(f"  - {model_name}: weight={weight:.2f}")

Enabled Models (5):
  - solar: weight=0.30
  - polyglot: weight=0.25
  - kullm: weight=0.20
  - kobart: weight=0.15
  - koalpaca: weight=0.10


In [3]:
# 로그 디렉토리 생성
# config의 로그 경로 사용
def get_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에 log_dir이 정의되어 있으면 사용, 없으면 기본값
if 'log_dir' in config['paths']:
    log_dir = get_path(config['paths']['log_dir'])
else:
    # 기본값: notebook_dir/logs/multi_model
    log_dir = notebook_dir / 'logs' / 'multi_model'

log_dir.mkdir(parents=True, exist_ok=True)

# 타임스탬프 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 로거 초기화
log_file = log_dir / f'ensemble_{len(enabled_models)}models_{timestamp}.log'
logger = NotebookLogger(
    log_path=str(log_file),
    print_also=True
)

logger.write('='*50)
logger.write('Multi-Model Ensemble Experiment')
logger.write(f'Timestamp: {timestamp}')
logger.write(f'Models: {enabled_models}')
logger.write(f'Ensemble Method: {config["ensemble_strategy"]["method"]}')
logger.write('='*50)

Multi-Model Ensemble Experiment
Timestamp: 20251010_090607
Models: ['solar', 'polyglot', 'kullm', 'kobart', 'koalpaca']
Ensemble Method: weighted_average


In [4]:
# TTA 설정 확인
if config['tta']['enabled']:
    logger.write("\nTTA Configuration:")
    logger.write(f"  - Augmentations: {config['tta']['num_augmentations']}")
    logger.write(f"  - Aggregation: {config['tta']['aggregation']}")
    
    for technique, settings in config['tta']['techniques'].items():
        if settings['enabled']:
            logger.write(f"  - {technique}: enabled")
            print(f"TTA: {technique} enabled")


TTA Configuration:
  - Augmentations: 3
  - Aggregation: mean
  - paraphrase: enabled
TTA: paraphrase enabled
  - reorder: enabled
TTA: reorder enabled


In [5]:
# GPU 체크 및 멀티 GPU 설정
if torch.cuda.is_available():
    gpu_tier = check_gpu_tier()
    logger.write(f"\nGPU: {torch.cuda.get_device_name(0)}")
    logger.write(f"GPU Tier: {gpu_tier}")
    
    # 멀티 GPU 체크
    if config['gpu']['multi_gpu']['enabled'] and torch.cuda.device_count() > 1:
        logger.write(f"Multi-GPU available: {torch.cuda.device_count()} GPUs")
    
    # 메모리 관리 설정
    if config['gpu']['empty_cache_between_models']:
        logger.write("Will clear GPU cache between models")


GPU: NVIDIA GeForce RTX 4090
GPU Tier: LOW
Will clear GPU cache between models


In [6]:
# 앙상블 가중치 시각화
weights = [config['ensemble_models'][name]['weight'] for name in enabled_models]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

plt.figure(figsize=(12, 6))
bars = plt.bar(enabled_models, weights, color=colors[:len(enabled_models)])
plt.title('Ensemble Model Weights Distribution', fontsize=14, fontweight='bold')
plt.ylabel('Weight', fontsize=12)
plt.xlabel('Model', fontsize=12)

# 가중치 값 표시
for bar, weight in zip(bars, weights):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{weight:.2f}', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

# 시각화 저장 - config 경로 사용
if 'visualization_dir' in config['paths']:
    viz_dir = get_path(config['paths']['visualization_dir'])
else:
    # 기본값
    viz_dir = log_dir / 'visualizations'

viz_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(viz_dir / f'ensemble_weights_{timestamp}.png', dpi=100, bbox_inches='tight')
plt.show()

logger.write(f"Weight visualization saved to {viz_dir}")

Weight visualization saved to /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/logs/multi_model/visualizations


/tmp/ipykernel_109226/3820716939.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
# Solar API 비교 설정
if config['solar_api_comparison']['enabled']:
    logger.write("\nSolar API Comparison Settings:")
    logger.write(f"  - API Key: {config['solar_api_comparison']['api_key'][:10]}...")
    logger.write(f"  - Use as baseline: {config['solar_api_comparison']['use_as_baseline']}")
    logger.write(f"  - Include in ensemble: {config['solar_api_comparison']['include_in_ensemble']}")
    
    print("\nSolar API configured for comparison")


Solar API Comparison Settings:
  - API Key: up_rMJWNzz...
  - Use as baseline: True
  - Include in ensemble: False

Solar API configured for comparison


In [8]:
# Optuna 최적화 설정 (앙상블 가중치)
if config['optuna']['enabled']:
    logger.write("\nOptuna Weight Optimization:")
    logger.write(f"  - Trials: {config['optuna']['n_trials']}")
    logger.write(f"  - Study: {config['optuna']['study_name']}")
    logger.write(f"  - Metric: {config['optuna']['metric']}")
    
    import optuna
    print("Optuna configured for ensemble weight optimization")


Optuna Weight Optimization:
  - Trials: 50
  - Study: ensemble_weight_optimization
  - Metric: rouge_l
Optuna configured for ensemble weight optimization


## 모델별 학습 및 평가

각 모델을 개별적으로 학습하고 평가합니다.

In [9]:
# 데이터 로드
# config 파일의 경로 사용
def get_data_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에서 데이터 경로 가져오기
train_path = get_data_path(config['paths']['train_file'])
dev_path = get_data_path(config['paths']['dev_file'])
test_path = get_data_path(config['paths']['test_file'])

logger.write("\n=== Data Loading ===")
logger.write(f"Loading data from config paths:")
logger.write(f"  - Train: {train_path}")
logger.write(f"  - Dev: {dev_path}")
logger.write(f"  - Test: {test_path}")

# 데이터 로드
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

logger.write(f"\nData loaded successfully!")
logger.write(f"  - Train samples: {len(train_df)}")
logger.write(f"  - Dev samples: {len(dev_df)}")
logger.write(f"  - Test samples: {len(test_df)}")

# 데이터 샘플 출력
print("\nTrain data topics:")
print(train_df['topic'].value_counts().head(10))
print(f"\nFirst dialogue (200 chars):")
print(train_df.iloc[0]['dialogue'][:200] + "...")


=== Data Loading ===
Loading data from config paths:
  - Train: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/train.csv
  - Dev: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/dev.csv
  - Test: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/test.csv

Data loaded successfully!
  - Train samples: 12457
  - Dev samples: 499
  - Test samples: 499

Train data topics:
topic
음식 주문     130
취업 면접     109
길 안내       66
호텔 체크인     40
아파트 임대     30
일상 대화      29
쇼핑         27
주말 계획      26
면접         25
호텔 예약      25
Name: count, dtype: int64

First dialogue (200 chars):
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person...


In [10]:
# 모델별 결과 저장
model_results = {}

for model_name in enabled_models:
    logger.write(f"\n=== {model_name} Model ===")
    model_config = config['ensemble_models'][model_name]
    logger.write(f"  - Model: {model_config['name']}")
    logger.write(f"  - Weight: {model_config['weight']}")
    
    if model_config.get('use_lora', False):
        logger.write(f"  - LoRA: r={model_config['lora_config']['r']}, alpha={model_config['lora_config']['alpha']}")
    
    # 실제 학습 코드는 여기에 구현
    print(f"Would train {model_name} here...")


=== solar Model ===
  - Model: upstage/SOLAR-10.7B-Instruct-v1.0
  - Weight: 0.3
  - LoRA: r=16, alpha=32
Would train solar here...

=== polyglot Model ===
  - Model: EleutherAI/polyglot-ko-12.8b
  - Weight: 0.25
  - LoRA: r=8, alpha=16
Would train polyglot here...

=== kullm Model ===
  - Model: nlpai-lab/kullm-v2
  - Weight: 0.2
  - LoRA: r=8, alpha=16
Would train kullm here...

=== kobart Model ===
  - Model: digit82/kobart-summarization
  - Weight: 0.15
Would train kobart here...

=== koalpaca Model ===
  - Model: beomi/KoAlpaca-Polyglot-12.8B
  - Weight: 0.1
  - LoRA: r=8, alpha=16
Would train koalpaca here...


## TTA (Text Test Augmentation)

텍스트 증강을 통한 성능 향상

In [11]:
if config['tta']['enabled']:
    logger.write("\n=== TTA Implementation ===")
    
    # Paraphrase
    if config['tta']['techniques']['paraphrase']['enabled']:
        logger.write("Paraphrase augmentation enabled")
        logger.write(f"  - Model: {config['tta']['techniques']['paraphrase']['model']}")
        logger.write(f"  - Variants: {config['tta']['techniques']['paraphrase']['num_variants']}")
    
    # 실제 TTA 구현은 여기에
    print("TTA would be applied here...")


=== TTA Implementation ===
Paraphrase augmentation enabled
  - Model: lcw99/t5-base-korean-paraphrase
  - Variants: 2
TTA would be applied here...


## 앙상블 및 최종 예측

모든 모델의 예측을 앙상블하여 최종 결과를 생성합니다.